Robot Localization with Python and Particle Filters
===================================================

Import libraries and load map.

In [1]:
import numpy as np
import cv2

# Load the map
map = cv2.imread("map.png", 0)
HEIGHT, WIDTH = map.shape

# Initialize robot state
rx, ry, rtheta = (WIDTH / 4, HEIGHT / 4, 0)

Map coordinate system

![title](images/coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

In [2]:
# Movement parameters
STEP = 5
TURN = np.radians(25)
SIGMA_STEP = 0.5
SIGMA_TURN = np.radians(5)

# Particle filter parameters
NUM_PARTICLES = 3000
SIGMA_SENSOR = 5
SIGMA_PARTICLE_STEP = 2
SIGMA_PARTICLE_TURN = np.pi / 24

# Function to handle keyboard input
def get_input():
    fwd = 0
    turn = 0
    halt = False
    k = cv2.waitKey(0) & 0xFF

    if k == ord('w'):  # Move forward
        fwd = STEP
    elif k == ord('s'):  # Move backward
        fwd = -STEP
    elif k == ord('d'):  # Turn right
        turn = TURN
    elif k == ord('a'):  # Turn left
        turn = -TURN
    elif k == ord('q'):  # Quit program
        halt = True
    else:
        print("Invalid input. Use W/A/S/D to move or Q to quit.")

    return fwd, turn, halt

Move the robot, with Gausssian noise.

![title](images/gaussian.png)

In [3]:
# Move robot with Gaussian noise
def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = fwd + np.random.normal(0.0, SIGMA_STEP)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)

    turn_noisy = turn + np.random.normal(0.0, SIGMA_TURN)
    rtheta += turn_noisy
    rtheta %= 2 * np.pi  # Keep angle within 0 to 2π

    return rx, ry, rtheta

Initialize particle cloud.

In [4]:
# Initialize particles
def init_particles():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array((WIDTH, HEIGHT, np.radians(360)))
    return particles

Move the particles.

In [5]:
# Move particles
def move_particles(particles, fwd, turn):
    particles[:, 0] += fwd * np.cos(particles[:, 2])
    particles[:, 1] += fwd * np.sin(particles[:, 2])
    particles[:, 2] += turn

    particles[:, 0] = np.clip(particles[:, 0], 0, WIDTH - 1)
    particles[:, 1] = np.clip(particles[:, 1], 0, HEIGHT - 1)
    particles[:, 2] %= 2 * np.pi  # Keep angles within 0 to 2π
    return particles

Get value from robot's sensor.

In [6]:
# Sense the map value
def sense(x, y, noisy=False):
    x, y = int(x), int(y)
    if noisy:
        return map[y, x] + np.random.normal(0.0, SIGMA_SENSOR)
    return map[y, x]

Compute particle weights.

In [7]:
# Compute particle weights
def compute_weights(particles, robot_sensor):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        particle_sensor = sense(particles[i, 0], particles[i, 1])
        errors[i] = abs(robot_sensor - particle_sensor)
    weights = np.max(errors) - errors

    # Set weights to zero for particles on the map edge
    weights[(particles[:, 0] == 0) |
            (particles[:, 0] == WIDTH - 1) |
            (particles[:, 1] == 0) |
            (particles[:, 1] == HEIGHT - 1)] = 0.0

    weights = weights ** 3  # Increase sensitivity
    return weights

Resample the particles.

In [8]:
# Resample particles based on weights
def resample(particles, weights):
    probabilities = weights / np.sum(weights)
    new_indices = np.random.choice(NUM_PARTICLES, size=NUM_PARTICLES, p=probabilities)
    particles = particles[new_indices, :]
    return particles

Add noise to the particles.

In [9]:
# Add noise to particles
def add_noise(particles):
    noise = np.concatenate((
        np.random.normal(0, SIGMA_PARTICLE_STEP, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_PARTICLE_STEP, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_PARTICLE_TURN, (NUM_PARTICLES, 1)),
    ), axis=1)
    particles += noise
    return particles

Display robot, particles and best guess.

In [12]:
# Display the map, particles, and robot with a resizable window
def display(map, rx, ry, particles, window_size=(800, 600)):

    # Calculate aspect ratios
    map_aspect = map.shape[1] / map.shape[0]
    window_aspect = window_size[0] / window_size[1]

    # Preserve aspect ratio when resizing
    if map_aspect > window_aspect:
        # Map is wider; fit width and adjust height
        new_width = window_size[0]
        new_height = int(window_size[0] / map_aspect)
    else:
        # Map is taller; fit height and adjust width
        new_height = window_size[1]
        new_width = int(window_size[1] * map_aspect)

    # Resize map to fit within the window while maintaining aspect ratio
    resized_map = cv2.resize(map, (new_width, new_height), interpolation=cv2.INTER_AREA)
    resized_map = cv2.cvtColor(resized_map, cv2.COLOR_GRAY2BGR)

    # Compute scaling factors
    scale_x = new_width / map.shape[1]
    scale_y = new_height / map.shape[0]

    # Draw particles
    for i in range(NUM_PARTICLES):
        px = int(particles[i, 0] * scale_x)
        py = int(particles[i, 1] * scale_y)
        cv2.circle(resized_map, (px, py), 1, (255, 0, 0), 1)

    # Draw robot
    robot_x = int(rx * scale_x)
    robot_y = int(ry * scale_y)
    cv2.circle(resized_map, (robot_x, robot_y), 5, (0, 255, 0), -1)

    # Draw best guess (mean position of particles)
    if len(particles) > 0:
        px_mean = int(np.mean(particles[:, 0]) * scale_x)
        py_mean = int(np.mean(particles[:, 1]) * scale_y)
        cv2.circle(resized_map, (px_mean, py_mean), 5, (0, 0, 255), -1)

    # Create a black background for the resized map to center it in the window
    display_image = np.zeros((window_size[1], window_size[0], 3), dtype=np.uint8)
    offset_x = (window_size[0] - new_width) // 2
    offset_y = (window_size[1] - new_height) // 2
    display_image[offset_y:offset_y + new_height, offset_x:offset_x + new_width] = resized_map

    # Display the image
    cv2.imshow('map', display_image)


Main routine.

In [13]:
# Main routine
particles = init_particles()
while True:
    display(map, rx, ry, particles, window_size=(1024, 768))
    fwd, turn, halt = get_input()
    if halt:
        print("Exiting program...")
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)

cv2.destroyAllWindows()               


Exiting program...
